### <p style="text-align: center; margin: 0;"> Міністерство освіти і науки України </p>
### <p style="text-align: center; margin: 0;"> Національний технічний університет України </p>
### <p style="text-align: center; margin: 0;"> «Київський політехнічний інститут імені Ігоря Сікорського» </p>
### <p style="text-align: center; margin: 0;"> «Інститут прикладного системного аналізу» </p>
### <p style="text-align: center; margin: 0;"> Кафедра математичних методів системного аналізу </p>
<br>

## <p style="text-align: center; margin: 0;"> Звіт </p>
### <p style="text-align: center; margin: 0;"> про виконання лабораторної роботи №2 </p>
### <p style="text-align: center; margin: 0;"> з дисципліни </p>
### <p style="text-align: center; margin: 0;"> «Розпізнавання образів» </p>
<br>

### <p style="text-align: right; margin: 0;"> Виконали: студенти ІV курсу &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> групи  КА-76  &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> Панасюк Я.І. &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> та групи  КА-74  &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> Іванов С.І. &emsp;&emsp;</p>
### <p style="text-align: right; margin: 0;"> Перевірила: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;</p>
### <p style="text-align: right; margin: 0;"> Дідковська М.В. &emsp;&emsp;</p>
<br>

### <p style="text-align: center; margin: 0;"> Київ – 2020 </p>
<br>
<br>

<br>

# Завдання

Для початку реалізувати енкодер та декодер (плеєр) для відеоряду на основі методів оцінки руху. Відповідно вхідним файлом буде відеозапис (наприклад трейлер до Тенет), одним з виходів буде файлик у вашому форматі що містить гіпотетично стиснений відеоряд з наприклад інформації щодо руху кожного третього кадру і два сусідні у повному розмірі (MPEG-7 алгоритми з що ми їх дотично згадували в попередніх лекціях будуть в нагоді). Другим виходом буде власне відтворення відео з вашого файлу. Після того як ви налаштували ваш енкодер і плеєр, йдете до друга по команді що в цей час налаштовував свої енкодер і плеєр на основі іншого алгоритму апроксимації руху і ви починаєте об'єднувати зусилля: користуючись вашими енкодерами ви генеруєте ваші апроксимовані зображення та обчислюєте їх різницю в порівнянні з дійсним зображеннями з оригінального відеоряду, збираючи при цьому візуальні різниці для кожного з методів. Окрім цього слід зібрати метрику швидкодії енкодерів і плеєрів, порівняти отримані результати і залляти їх аналогічним чином в гуглдок. Бонусні бали за опцію datamoshing і запис та відтворення аудіо, підтримку більш ніж одного формату відео, сприйняття стрімів з Youtube.

<br>

# Хід роботи

## Алгоритми апроксимації руху

### Farneback method opt.flow calculation

Encoder

In [17]:
import numpy as np
import cv2
import time

# Get our sample video
cap = cv2.VideoCapture('walking.avi')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('Farn_encode.avi',fourcc, 20.0, (frame_width,frame_height))

# we will aproximate only even frames, while odd frms won't be change
odd_frame = True

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

t_start = time.time()
while(cap.isOpened()):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Create an image representing our opt.flow
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)    
    frame_flow_rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    cv2.imshow('actual footage', frame2)
    cv2.imshow('pure optical flow',frame_flow_rgb)
    
    # write frame to output video.
    if(odd_frame):
        out.write(frame2)
        odd_frame = False
    else:
        out.write(frame_flow_rgb)
        odd_frame = True
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    # Aaaaand... go to the next frame!
    prvs = next

t_end = time.time()
# time metric
Farn_encode_time = t_end - t_start

cap.release()
out.release()
cv2.destroyAllWindows()

Decoder

In [27]:
import numpy as np
import cv2

# Get our sample video
cap = cv2.VideoCapture('walking.avi')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('Farn_decode.avi',fourcc, 20.0, (frame_width,frame_height))

# we will aproximate only even frames, while odd frms won't be change
odd_frame = True

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

t_start = time.time()
while(cap.isOpened()):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 30, 3, 5, 1.2, 0)
    
    h, w = flow.shape[:2]
    flow = -flow # here we invert the flow in order to get the previous image
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]

    frame_warped = cv2.remap(frame2, flow, None, cv2.INTER_LINEAR)
    
    cv2.imshow('warped', frame_warped)
    
    if(odd_frame):
        out.write(frame2)
        odd_frame = False
    else:
        out.write(frame_warped)
        odd_frame = True
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Aaaaand... go to the next frame!
    prvs = next

t_end = time.time()
# time metric
Farn_decode_time = t_end - t_start
    
cap.release()
out.release()
cv2.destroyAllWindows()

<br>

### Dual TV L1 method opt.flow calculation

Encoder

In [22]:
import numpy as np
import cv2

# Get our sample video
cap = cv2.VideoCapture('walking.avi')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('Dual_encode.avi',fourcc, 20.0, (frame_width,frame_height))

# we will aproximate only even frames, while odd frms won't be change
odd_frame = True

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

t_start = time.time()
while(cap.isOpened()):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = optical_flow.calc(prvs, next, None)
    
    # Create an image representing our opt.flow
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)    
    frame_flow_rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    cv2.imshow('actual footage', frame2)
    cv2.imshow('pure optical flow',frame_flow_rgb)
    
    # write frame to output video.
    if(odd_frame):
        out.write(frame2)
        odd_frame = False
    else:
        out.write(frame_flow_rgb)
        odd_frame = True
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    # Aaaaand... go to the next frame!
    prvs = next
    
t_end = time.time()
# time metric
Dual_encode_time = t_end - t_start

cap.release()
out.release()
cv2.destroyAllWindows()

Decoder

In [28]:
import numpy as np
import cv2

# Get our sample video
cap = cv2.VideoCapture('walking.avi')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('Dual_decode.avi',fourcc, 20.0, (frame_width,frame_height))
odd_frame = True

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

t_start = time.time()
while(cap.isOpened()):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = optical_flow.calc(prvs, next, None)
    
    h, w = flow.shape[:2]
    flow = -flow # here we invert the flow in order to get the previous image
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]

    frame_warped = cv2.remap(frame2, flow, None, cv2.INTER_LINEAR)
    
    cv2.imshow('warped', frame_warped)
    
    if(odd_frame):
        out.write(frame2)
        odd_frame = False
    else:
        out.write(frame_warped)
        odd_frame = True
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Aaaaand... go to the next frame!
    prvs = next

t_end = time.time()
# time metric
Dual_decode_time = t_end - t_start    
    
cap.release()
out.release()
cv2.destroyAllWindows()

<br>

## Порівняння алгоритмів

Точність

Швидкість роботи

In [ ]:
Farn_encode_time

In [ ]:
Farn_decode_time

In [ ]:
Dual_encode_time

In [ ]:
Dual_decode_time

<br>